<a href="https://colab.research.google.com/github/hasankarabela/online_payment_fraud_detection/blob/main/online_payment_fraud_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import pandas as pd
import numpy as np
data = pd.read_csv("credit_card.csv")
print(data.head())

   step      type    amount     nameOrig  oldbalanceOrg  newbalanceOrig  \
0     1   PAYMENT   9839.64  C1231006815       170136.0       160296.36   
1     1   PAYMENT   1864.28  C1666544295        21249.0        19384.72   
2     1  TRANSFER    181.00  C1305486145          181.0            0.00   
3     1  CASH_OUT    181.00   C840083671          181.0            0.00   
4     1   PAYMENT  11668.14  C2048537720        41554.0        29885.86   

      nameDest  oldbalanceDest  newbalanceDest  isFraud  isFlaggedFraud  
0  M1979787155             0.0             0.0      0.0             0.0  
1  M2044282225             0.0             0.0      0.0             0.0  
2   C553264065             0.0             0.0      1.0             0.0  
3    C38997010         21182.0             0.0      1.0             0.0  
4  M1230701703             0.0             0.0      0.0             0.0  


In [42]:
print(data.isnull().sum())

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    1
isFraud           1
isFlaggedFraud    1
dtype: int64


In [43]:
data = data.dropna()
data.info()


<class 'pandas.core.frame.DataFrame'>
Index: 1172839 entries, 0 to 1172838
Data columns (total 11 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   step            1172839 non-null  int64  
 1   type            1172839 non-null  object 
 2   amount          1172839 non-null  float64
 3   nameOrig        1172839 non-null  object 
 4   oldbalanceOrg   1172839 non-null  float64
 5   newbalanceOrig  1172839 non-null  float64
 6   nameDest        1172839 non-null  object 
 7   oldbalanceDest  1172839 non-null  float64
 8   newbalanceDest  1172839 non-null  float64
 9   isFraud         1172839 non-null  float64
 10  isFlaggedFraud  1172839 non-null  float64
dtypes: float64(7), int64(1), object(3)
memory usage: 107.4+ MB


In [45]:
print(data.type.value_counts())

type
CASH_OUT    415313
PAYMENT     397439
CASH_IN     254778
TRANSFER     97225
DEBIT         8084
Name: count, dtype: int64


In [46]:
type = data["type"].value_counts()
transactions = type.index
quantity = type.values

import plotly.express as px
figure = px.pie(data,
             values=quantity,
             names=transactions,hole = 0.3,
             title="Distribution of Transaction Type")
figure.show()

In [47]:
data1 = data.select_dtypes(include=[np.number])
correlation = data1.corr()
print(correlation["isFraud"].sort_values(ascending=False))

isFraud           1.000000
amount            0.140200
step              0.027932
oldbalanceOrg     0.004686
newbalanceDest   -0.000160
oldbalanceDest   -0.008036
newbalanceOrig   -0.010250
isFlaggedFraud         NaN
Name: isFraud, dtype: float64


In [49]:
from sklearn.model_selection import train_test_split
print(data.columns)
data["type"] = data["type"].map({"CASH_OUT": 1, "PAYMENT": 2,
                                 "CASH_IN": 3, "TRANSFER": 4,
                                 "DEBIT": 5})
data["isFraud"] = data["isFraud"].map({0.0: "No Fraud", 1.0: "Fraud"})
print(data.head())
x = np.array(data[["type", "amount", "oldbalanceOrg", "newbalanceOrig"]])
y = np.array(data[["isFraud"]])

Index(['step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg', 'newbalanceOrig',
       'nameDest', 'oldbalanceDest', 'newbalanceDest', 'isFraud',
       'isFlaggedFraud'],
      dtype='object')
   step  type    amount     nameOrig  oldbalanceOrg  newbalanceOrig  \
0     1   NaN   9839.64  C1231006815       170136.0       160296.36   
1     1   NaN   1864.28  C1666544295        21249.0        19384.72   
2     1   NaN    181.00  C1305486145          181.0            0.00   
3     1   NaN    181.00   C840083671          181.0            0.00   
4     1   NaN  11668.14  C2048537720        41554.0        29885.86   

      nameDest  oldbalanceDest  newbalanceDest   isFraud  isFlaggedFraud  
0  M1979787155             0.0             0.0  No Fraud             0.0  
1  M2044282225             0.0             0.0  No Fraud             0.0  
2   C553264065             0.0             0.0     Fraud             0.0  
3    C38997010         21182.0             0.0     Fraud             0.0  
4  M

In [52]:
from sklearn.tree import DecisionTreeClassifier
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.15, random_state=42)
model = DecisionTreeClassifier()
model.fit(xtrain, ytrain)
print(model.score(xtest, ytest))

0.9996646317201551


In [53]:
features = np.array([[4, 9000.60, 9000.60, 0.0]])
print(model.predict(features))

['Fraud']
